## 2 Layer MLP for Dark Knoweldge Experiments w/AlignMNIST

75 epochs of RMSProp

Eval extra data first, on same size network

100 runs

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
%run augmentmnist.py

In [ ]:
num_epochs = 75
batch_size = 125

learning_rate = 0.001

In [ ]:
# Network Parameters
n_hidden_1 = 800 # 1st layer num features
n_hidden_2 = 800 # 2nd layer num features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

std_0  = np.sqrt(6.0/(n_input+n_hidden_1))
std_h1 = np.sqrt(6.0/(n_hidden_1+n_hidden_2))
std_h2 = np.sqrt(6.0/(n_hidden_2+n_classes))

logfile = "2-layer-dlk-alignmnist"

In [ ]:
# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

xt = tf.placeholder("float", [None, n_input])
yt = tf.placeholder("float", [None, n_classes])

In [ ]:
def multilayer_perceptron(_X, _weights, _biases):
    layer_1 = tf.nn.relu(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])) 
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, _weights['h2']), _biases['b2'])) 
    return tf.matmul(layer_2, weights['out']) + biases['out']

In [ ]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=std_0)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=std_h1)),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes], stddev=std_h2))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1], stddev=0.1)),
    'b2': tf.Variable(tf.random_normal([n_hidden_2], stddev=0.01)),
    'out': tf.Variable(tf.random_normal([n_classes], stddev=0.001))
}

In [ ]:
mlp = multilayer_perceptron(x, weights, biases )
mlp_test = multilayer_perceptron(xt, weights, biases )

In [ ]:
cost =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(mlp, y)) 
gs = tf.get_variable("global_step",[],trainable=False,initializer=tf.constant_initializer(0))
lr = tf.constant(learning_rate) 

### Original MNIST Data

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True)
trX_0, trY_0 = mnist.train.images, mnist.train.labels
teX_0, teY_0 = mnist.test.images, mnist.test.labels

In [ ]:
test_accuracies = []
train_accuracies = []
for i in range(25):
    alignmnist = AlignMNIST()
    train_op = tf.contrib.layers.optimize_loss(cost, global_step=gs, learning_rate=lr,optimizer="RMSProp")

    pred = tf.equal(tf.argmax(mlp, 1), tf.argmax(y, 1)) # Count correct predictions
    train_acc_op = tf.reduce_mean(tf.cast(pred, "float"))  # Cast boolean to float to average

    test_pred = tf.equal(tf.argmax(mlp_test, 1), tf.argmax(yt, 1)) # Count correct predictions
    test_acc_op = tf.reduce_mean(tf.cast(test_pred, "float"))  # Cast boolean to float to average

    with tf.Session() as sess:
        
        tf.initialize_all_variables().run()

        for epoch in range(num_epochs):
            trX, trY = alignmnist.next_epoch()

            for start, end in zip(range(0, len(trX), batch_size), range(batch_size, len(trX), batch_size)):
                sess.run(train_op, feed_dict={x: trX[start:end], y: trY[start:end]})

            trn_acc, tst_acc = sess.run([train_acc_op, test_acc_op], feed_dict={x: trX, y: trY, xt: teX_0, yt: teY_0})
            
            train_accuracies.append(trn_acc)
            test_accuracies.append(tst_acc)
     

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

In [ ]:
plt.plot(train_accuracies)
plt.plot(test_accuracies)

In [ ]:
np.max(test_accuracies)

Number test errors is

In [ ]:
10000*(1.0-np.max(test_accuracies))

#### this sets the baseline improvement, with better data, from 146 to 103

I expect aggressive dropout to improve from 103 to ~70

and i have yet to figure out how to make this happen
